In [1]:
import math
import numpy as np
import openrtdynamics2.lang as dy
import openrtdynamics2.py_execute as dyexe

In [8]:
cpp_class_code = """

class SomeClass {
public:

    int x;
    
    void reset() {
        x = 0;
    }
    
    void return_output_signals(double & output1, double & output2, double input) {
    
        output1 = input;
        output2 = 2.0 * x;
        
    }
    
    void update_states(double input) {
        x = x + input;
    }

};

typedef SomeClass *SomeClassPtr;
"""

In [9]:
dy.clear()

system = dy.enter_system()


class_ptr_datatype = dy.DataTypePointer( cpp_ptr_type_name = 'SomeClassPtr' )  
class_ptr = dy.cpp_allocate_class( datatype=class_ptr_datatype, code_constructor_call='new SomeClass()' )

class_ptr.set_name('pointer').set_blockname('pointer_source')

u = dy.float64(1.0)

outputs = dy.cpp_call_class_member_function(
    ptr_signal = class_ptr,

    input_signals=[ u ],
    input_types=[ dy.DataTypeFloat64(1) ],
    output_types=[ dy.DataTypeFloat64(1), dy.DataTypeFloat64(1) ],

    member_function_name_to_calc_outputs  = 'return_output_signals',
    member_function_name_to_update_states = 'update_states'
)

output1 = outputs[0]
output2 = outputs[1]

# define output(s)
dy.append_primay_ouput(output1, 'output1')
dy.append_primay_ouput(output2, 'output2')

# generate code for Web Assembly (wasm), requires emcc (emscripten) to build
code_gen_results = dy.generate_code(template=dy.WasmRuntime(), folder="generated/delay", build=False, include_code_list=[cpp_class_code])

compiling system simulation (level 0)... 
determining the computation order...
building execution paths...
All dependencies are resolved.
Generated code will be written to generated/delay .


In [10]:
# show the generated source code
from IPython.display import Code
Code(data=code_gen_results['algorithm_sourcecode'], language='c++')

// custom code


class SomeClass {
public:

    int x;
    
    void reset() {
        x = 0;
    }
    
    void return_output_signals(double & output1, double & output2, double input) {
    
        output1 = input;
        output2 = 2.0 * x;
        
    }
    
    void update_states(double input) {
        x = x + input;
    }

};

typedef SomeClass *SomeClassPtr;

// end of custom code

// namespace for simulation {
  // global variables

  class simulation {
    public:


    // state update
    SomeClassPtr block_pointer_source_class = new SomeClass();


    //
    // cached output values
    //

    double s2__block_2;
    SomeClassPtr s1_pointer__block_pointer_source;
    double output1__block_3;
    double output2__block_3;

    // API-function resetStates
    void resetStates() { // created by cpp_define_function

      block_pointer_source_class->reset();
    }
    // output signals of  resetStates
    struct Outputs_resetStates{
      ;

    };
    // input signals of resetStates
    struct Inputs_resetStates{
      ;

    };
    // wrapper function for resetStates
    Outputs_resetStates resetStates__(Inputs_resetStates inputs) {
      Outputs_resetStates outputs;

      resetStates();

      return outputs;
    }
    // API-function updateStates
    void updateStates() { // created by cpp_define_function


      // restoring the signals s2, s1_pointer, output1, output2 from the states 
      double &s2 = s2__block_2;
      SomeClassPtr &s1_pointer = s1_pointer__block_pointer_source;
      double &output1 = output1__block_3;
      double &output2 = output2__block_3;


      // calculating the block outputs in the following order 
      // that depend on 
      // dependencies that require a state update are  


      { // calling the custom c++ function s1_pointer->update_states
        s1_pointer->update_states(s2);
      }
    }
    // output signals of  updateStates
    struct Outputs_updateStates{
      ;

    };
    // input signals of updateStates
    struct Inputs_updateStates{
      ;

    };
    // wrapper function for updateStates
    Outputs_updateStates updateStates__(Inputs_updateStates inputs) {
      Outputs_updateStates outputs;

      updateStates();

      return outputs;
    }
    // API-function calcResults_1 to compute: output1, output2
    void calcResults_1(double &output1, double &output2) { // created by cpp_define_function
      double s2;
      SomeClassPtr s1_pointer;


      // calculating the block outputs in the following order s2, s1_pointer, output1, output2
      // that depend on 
      // dependencies that require a state update are s2, s1_pointer 

      s2 = 1.0;
      s1_pointer = block_pointer_source_class;
      { // calling the custom c++ function s1_pointer->return_output_signals
        double block_3_out0;
        double block_3_out1;
        s1_pointer->return_output_signals(block_3_out0, block_3_out1, s2);
        output1 = block_3_out0;
        output2 = block_3_out1;
      }

      // saving the signals s2, s1_pointer, output1, output2 into the states 
      s2__block_2 = s2;
      s1_pointer__block_pointer_source = s1_pointer;
      output1__block_3 = output1;
      output2__block_3 = output2;
    }
    // output signals of  calcResults_1
    struct Outputs_calcResults_1{
      double output1;
      double output2;

    };
    // input signals of calcResults_1
    struct Inputs_calcResults_1{
      ;

    };
    // wrapper function for calcResults_1
    Outputs_calcResults_1 calcResults_1__(Inputs_calcResults_1 inputs) {
      Outputs_calcResults_1 outputs;

      calcResults_1(outputs.output1, outputs.output2);

      return outputs;
    }
    // all system inputs and outputs combined
    struct Inputs{
      ;

    };
    struct Outputs{
      double output1;
      double output2;

    };
    // main step function 
    void step(Outputs & outputs, Inputs const & inputs, int calculate_outputs, bool update_states, bool reset_states) {
      if (reset_states) {
   

In [11]:
compiled_system = dyexe.CompiledCode(code_gen_results)
testsim = dyexe.SystemInstance(compiled_system)

In [12]:
sim_results = dyexe.run_batch_simulation(testsim, input_data={}, N=10 )
sim_results['output1']

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [13]:
sim_results['output2']

array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.])